In [1]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-04-17 12:47:46,753	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-17 12:47:49,456	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import gdown

gdown.download("https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t")
data_path = "combo_sciplex_prep_hvg_filtered.h5ad"
adata = sc.read(data_path)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t
From (redirected): https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t&confirm=t&uuid=1e1aced8-822d-4f26-9bdc-5b96a69106ae
To: /home/icb/dominik.klein/git_repos/ot_pert_new/competing_methods/combosciplex/CPA/combo_sciplex_prep_hvg_filtered.h5ad
100%|██████████| 490M/490M [00:07<00:00, 69.7MB/s] 


In [4]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_train.obs["split"].value_counts()

split
train    51882
Name: count, dtype: int64

In [7]:
adata_test.obs["split"].value_counts()

split
test    3100
Name: count, dtype: int64

In [8]:
adata_ood.obs["split"].value_counts()

split
ood    8896
Name: count, dtype: int64

In [9]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

In [10]:
adata_combined.X = adata_combined.layers["counts"]

In [11]:
adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

In [12]:
adata_combined.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_combined.obs["condition_ID"] = adata.obs["condition_ID"]
adata_combined.obs["log_dose"] = adata.obs["log_dose"]

In [13]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [14]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [15]:
adata_combined.obs["split_1ct_MEC"].value_counts()

split_1ct_MEC
train_train    46592
ood             8896
train_valid     5290
test            3100
Name: count, dtype: int64

In [16]:
adata_combined.obs["condition_ID"] = adata_combined.obs["condition_ID"].cat.remove_unused_categories()

In [17]:
cpa.CPA.setup_anndata(
    adata_combined,
    perturbation_key="condition_ID",
    dosage_key="log_dose",
    control_group="CHEMBL504",
    batch_key=None,
    is_count_data=True,
    categorical_covariate_keys=["cell_type"],
    # deg_uns_key='rank_genes_groups_cov',
    deg_uns_cat_key="cov_drug_dose",
    max_comb_len=2,
)

100%|██████████| 63878/63878 [00:00<00:00, 1146722.78it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [18]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "nb",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [19]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [20]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex",
)

100%|██████████| 32/32 [00:00<00:00, 117.27it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:18<10:14:38, 18.48s/it, v_num=1, recon=322, r2_mean=0.758, adv_loss=2.08, acc_pert=0.257]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [02:52<9:57:40, 18.01s/it, v_num=1, recon=304, r2_mean=0.77, adv_loss=1.89, acc_pert=0.307, val_recon=316, disnt_basal=0.068, disnt_after=0.0958, val_r2_mean=0.758, val_KL=nan] 


Epoch 00009: cpa_metric reached. Module best state updated.


...


disnt_basal = 0.05208604748984617
disnt_after = 0.07872925166271896
val_r2_mean = 0.7692172339255796
val_r2_var = 0.15227303015134416
Epoch 25/2000:   1%|          | 24/2000 [07:21<9:34:52, 17.46s/it, v_num=1, recon=281, r2_mean=0.778, adv_loss=1.83, acc_pert=0.339, val_recon=289, disnt_basal=0.0521, disnt_after=0.0787, val_r2_mean=0.769, val_KL=nan] 


Epoch 00024: cpa_metric reached. Module best state updated.


Epoch 30/2000:   1%|▏         | 29/2000 [08:50<9:34:05, 17.48s/it, v_num=1, recon=278, r2_mean=0.779, adv_loss=1.84, acc_pert=0.336, val_recon=285, disnt_basal=0.0505, disnt_after=0.0771, val_r2_mean=0.77, val_KL=nan]  
disnt_basal = 0.0475251429363868
disnt_after = 0.07428038706028171
val_r2_mean = 0.7691851369984745
val_r2_var = 0.14635948421516135
Epoch 40/2000:   2%|▏         | 39/2000 [11:40<8:54:20, 16.35s/it, v_num=1, recon=276, r2_mean=0.779, adv_loss=3.35, acc_pert=0.0615, val_recon=283, disnt_basal=0.0361, disnt_after=0.0614, val_r2_mean=0.766, val_KL=nan]
disnt_basal = 0.03542979716767703
disnt_after = 0.06468646208088279
val_r2_mean = 0.7676456539508342
val_r2_var = 0.1421720646321167
Epoch 50/2000:   2%|▏         | 49/2000 [14:28<8:51:45, 16.35s/it, v_num=1, recon=274, r2_mean=0.779, adv_loss=3.3, acc_pert=0.0511, val_recon=282, disnt_basal=0.0358, disnt_after=0.065, val_r2_mean=0.769, val_KL=nan]  
disnt_basal = 0.03355446014535945
disnt_after = 0.0630799317756727
val_r2_


Epoch 00054: cpa_metric reached. Module best state updated.


Epoch 60/2000:   3%|▎         | 59/2000 [17:22<8:58:47, 16.65s/it, v_num=1, recon=273, r2_mean=0.78, adv_loss=3.29, acc_pert=0.0478, val_recon=281, disnt_basal=0.0336, disnt_after=0.0636, val_r2_mean=0.772, val_KL=nan] 
disnt_basal = 0.03260389333169507
disnt_after = 0.06209424029269509
val_r2_mean = 0.7672058922964579
val_r2_var = 0.14131458446702447
Epoch 70/2000:   3%|▎         | 69/2000 [20:12<8:52:25, 16.54s/it, v_num=1, recon=272, r2_mean=0.782, adv_loss=3.29, acc_pert=0.0472, val_recon=281, disnt_basal=0.0324, disnt_after=0.0613, val_r2_mean=0.765, val_KL=nan]
disnt_basal = 0.033340752400508544
disnt_after = 0.06291656288889788
val_r2_mean = 0.7663209459200634
val_r2_var = 0.13975641035452221
Epoch 80/2000:   4%|▍         | 79/2000 [22:58<8:33:18, 16.03s/it, v_num=1, recon=271, r2_mean=0.782, adv_loss=3.28, acc_pert=0.0498, val_recon=280, disnt_basal=0.0326, disnt_after=0.0628, val_r2_mean=0.766, val_KL=nan]
disnt_basal = 0.03276622504018874
disnt_after = 0.06149718128680096
val

In [21]:
model.predict(adata_combined)

100%|██████████| 1997/1997 [00:06<00:00, 304.27it/s]


In [22]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions.h5ad"
)